## Ejercicio 2

### **Teoría**
#### **Defina y responda:**


 **1. Considere la taxonomía de rasgos, clasifique y justifique cada uno de los siguientes (discreto, continuo o relacional):**

 **a. Edad**
- A pesar de que se suele medir discretamente, en realidad es una variable **continua**. Esto se debe a que la edad puede tomar un valor numérico en un rango que no se limita a un conjunto discreto de categorías predeterminadas. Es decir, podemos expresar la edad en decimales (ej. 12.5 años), representando una cantidad medible.

**b. Profesión**
- Es una variable **discreta**, puesto que una profesión pertenece a un conjunto finito de categorías. Además, no existe una escala numérica que conecte estas categorías, por lo que se considera cualitativa.

**c. Redes de amistad**
- Esta es una variable **relacional**, porque representa explícitamente una conexión o vínculo entre individuos. Se puede modelar como un grafo, en donde los nodos representan a los agentes y las aristas indican las relaciones.

**d. Estado de vacunación**
- Representa una variable **discreta**, puesto que hay un conjunto limitado de posibles valores para representar el estado de vacunación (vacunado, no vacunado, parcalmente vacunado).

<br>

**2. Calcule el tamaño total del espacio de parámetros y responda ¿cómo podría afectar esto al tiempo de ejecución de la simulación?**

Para esto, considere un escenario de 10,000 agentes con:

**a. 3 rasgos continuos (p. ej., tasa de movilidad)**
- Para rasgos contínuos, teóricamente su espacio de parámetros tiene la posibilidad de tomar valores infinitos, ya que cada rasgo puede tomar cualquier valor real dentro de su rango definido. Sin embargo, en la práctica, se discretiza cada rasgo en $k_i$ niveles (dividir el rango en pasos uniformes).

Las combinaciones por los 3 rasgos por agente se calcularían como:


$$
k_c * k_c * k_c = k_c^3
$$

En el caso de ser 10,000 agentes, el número total de configuraciones posibles sería de:

$$
(k_c^3)^{10000}
$$

- En tiempos de ejecución, si aumentamos $k_i$ (la resolución), el número de configuraciones a probar,y por tanto el tiempo total de experimentación. Cada corrida además cuesta al menos $$ O(agentes * pasos) $$

**b. 2 rasgos discretos (p. ej., ocupación)**
- Sea $k_{d1}$ el número de categorías del primer rasgo discreto y $k_{d2}$ el del segundo

Las combinaciones por los 2 rasgos por agente se calcularían como:

$$
k_{d1} * k_{d2}
$$

De la misma manera, las asignaciones posibles para toda la población sería de:

$$
(k_{d1}* k_{d2})^{10000}
$$

- Si hablamos del tiempo de ejecución, a diferencia de los rasgos continuos, aquí el espacio de parámetros es finito. Esto significa que, aunque sigue creciendo conforme aumentan las categorías, lo hace de manera mucho más controlada, por lo que su tamaño crece de forma lineal con el número de valores posibles.

Sin embargo, al escalar la simulación a 10,000 agentes, el número de asignaciones posibles todavía se dispara, por lo que resulta inviable explorar absolutamente todas las configuraciones. Por eso, igual que en los rasgos continuos, lo común es muestrear parte del espacio de parámetros y trabajar con réplicas para capturar la variabilidad.


## **Práctica**

Construya un perfil poblacional heterogéneo para un escenario de “Respuesta a una pandemia urbana”, para ello
considere las siguientes tareas


**1. Defina una clase de Python con:**
<br>
**a. 3 rasgos discretos (p. ej., ocupación)**
<br>
**b. 2 rasgos continuos (p. ej., tasa de movilidad)**
<br>
**c. 1 rasgo relacional (p. ej., identificación del hogar)**
<br>
**d. Responda: ¿Cómo encodearías los trabajadores de la salud vs los maestros para evitar sesgo?**

- Para evitar sesgo al encodear trabajadores de la salud y maestros, lo importante es mantener la ocupación como una categoría neutral. La variable ocupacion debe servir únicamente como una etiqueta descriptiva (doctor, maestro, etc.), sin imponer de forma directa que una ocupación tiene más o menos riesgo que otra.

    El nivel de exposición debe modelarse a través de otros rasgos que reflejen el contexto real, como las horas de trabajo presencial, el tamaño de los grupos con los que interactúan, el uso de equipo de protección, si los turnos son en el día o en la noche, etc. Así, el riesgo no se asigna por el simple hecho de ser maestro o personal de salud, sino por las condiciones en las que se desempeña cada uno. De esta forma, las diferencias en probabilidad de contacto o transmisión emergen de factores observables y comparables, y no de supuestos subjetivos sobre la ocupación.

In [ ]:
from dataclasses import dataclass

@dataclass
class Agente:
    # --- Rasgos discretos ---
    ocupacion: str            # doctor, maestro, empleado, independiente, desempleado
    nivel_ingresos: str       # bajo,medio,alto
    esquema_vacunacion: str   # ninguna,incompleta,completa,refuerzo

    # --- Rasgos continuos ---
    tasa_movilidad: float     # desplazamientos/día (>=0)
    prob_contacto: float      # probabilidad de entablar contacto por encuentro (0–1)

    # --- Rasgo relacional ---
    comunidad_id: int         # identificador de comunidad o zona de la ciudad


**2. Proponer métodos de muestreo para distribuciones realistas (p. ej. Distrubución Gamma)**

- Para representar mejor la realidad, usaría diferentes distribuciones según el tipo de rasgo. Por ejemplo, la Distribución Gamma  sirve para variables como la movilidad diaria, porque muchas personas se mueven poco y unas pocas se mueven mucho. Para variables entre 0 y 1, como la probabilidad de contacto o el acceso a vacunas, conviene una Beta, que se puede ajustar a distintos promedios. En el caso de categorías como ocupación o vacunación, lo más adecuado es tomar probabilidades basadas en datos o porcentajes de la población.

    **a. Responda: Si los datos muestran que el 20% de las enfermeras trabajan en turnos de noche, ¿cómo
    validarías esto en tu modelo?**

    - Si quiero asegurar que el 20% de las enfermeras trabajan en turno de noche, en el modelo lo valido de manera simple: después de asignar a las personas que son enfermeras, selecciono exactamente un 20% y las marco con turno nocturno. Luego reviso que, en el total de enfermeras, la proporción de noche coincida con ese 20%. Con esto garantizo que el modelo respete el dato observado.


**3. Predecir cómo las correlaciones entre rasgos (p. ej., bajos ingresos + bajo acceso a vacunas) podrían
distorsionar la dinámica del brote. Fundamente su hipótesis.**

- Cuando dos rasgos como bajos ingresos y bajo acceso a vacunas se combinan, el brote puede cambiar bastante. Esto pasa porque en ciertos grupos se acumulan más personas susceptibles, ya sea porque viven en casas con muchas personas, usan más el transporte público o tienen menos posibilidad de vacunarse. Aunque en promedio la ciudad parezca tener buena cobertura, esos grupos generan focos locales de contagio donde el virus circula más tiempo.

    La consecuencia es que la epidemia no se comporta de manera uniforme: en algunas comunidades los casos bajan rápido, pero en otros siguen apareciendo brotes. Esto hace que la curva general sea más difícil de controlar y obliga a pensar en intervenciones dirigidas, no solo medidas iguales para todos.